<a href="https://colab.research.google.com/github/gopiprasanthpotipireddy/Quartic.ai-Challenge/blob/master/Quarticai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

installing Python utilities

In [3]:
!apt-get --fix-broken install python-pycurl python-apt
!apt-get install -y -qq software-properties-common module-init-tools git-core
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null


Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pycurl is already the newest version (7.43.0.1-0.2).
python-apt is already the newest version (1.6.3ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


installing and suthenticating drive

In [0]:
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

mounting drive

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

Quartic App Api

In [0]:
import os
import pandas as pd
from sklearn.metrics import average_precision_score,confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt

class QuarticApp:
    def __init__(self):
        self.train_data = None
        self.test_data= None
        return
    
    def read_data(self):
        file_location="drive/Quartic.ai/ds_data/"
        self.train_data=pd.read_csv(os.path.join(file_location,"data_train.csv"))
        self.test_data=pd.read_csv(os.path.join(file_location,"data_test.csv"))
        #train_data.info()
        #test_data=pd.read_csv(os.path.join(file_location,"data_test.csv"))
        #test_data.info()
        return

    def build_model(self,model_type):
        #model
        return
    
    def ValidateColumns(self):    #checking for imputed and non imputed columns
        self.features=list(self.train_data.columns)
        self.im_features=[]
        self.nonim_columns=[]
        for i in range(1,len(self.train_data.columns)):
            if self.train_data.iloc[:,i].isna().sum() == 0:
                self.nonim_columns.append(self.features[i])
            else:
                self.im_features.append(self.features[i])
                    
        return 

def roc_binary(y_test,y_predict):
    
    fpr, tpr,_ = roc_curve(y_test, y_predict)
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',
             lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show() 

reading data

In [0]:
Obj=QuarticApp()
Obj.read_data()
train_data=Obj.train_data
train_data.columns
train_data.info()
test_data=Obj.test_data

In [0]:
train_data=train_data.drop(['id'],axis=1)
test_data=test_data.drop(['id'],axis=1)


Validating for non imputable columns

In [0]:
Obj.ValidateColumns()

In [0]:
print (Obj.nonim_columns)
Obj.nonim_columns.remove('target')

x and y

In [0]:
x=train_data[Obj.nonim_columns]
y=train_data["target"]

installing imblearn package

In [11]:
!pip install -U git+https://github.com/scikit-learn-contrib/imbalanced-learn.git

  Cloning https://github.com/scikit-learn-contrib/imbalanced-learn.git to /tmp/pip-req-build-l2hcaes4
  Stored in directory: /tmp/pip-ephem-wheel-cache-tc7gjvvw/wheels/6c/07/cf/38cb9b7cc9e6a0ac7648a80ec192b6f2d863405fb0049ac0ff
Successfully built imbalanced-learn
  Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


In [12]:
from imblearn.over_sampling import RandomOverSampler

Using TensorFlow backend.


In [0]:
from imblearn import over_sampling

In [0]:
from sklearn.svm import LinearSVC
from sklearn import model_selection
from sklearn.metrics import average_precision_score,confusion_matrix,precision_recall_curve,f1_score

Preparing Augmented Data using Random OverSampling

In [0]:
#random oversampling
ros = RandomOverSampler(random_state=0)

x_resampled, y_resampled = ros.fit_resample(x, y)

Fitting Svc

In [0]:
cl=LinearSVC()


In [23]:
train_data.columns

Index(['num1', 'num2', 'num3', 'num4', 'num5', 'num6', 'num7', 'num8', 'num9',
       'num10', 'num11', 'num12', 'num13', 'num14', 'num15', 'num16', 'num17',
       'num18', 'num19', 'num20', 'num21', 'num22', 'num23', 'der1', 'der2',
       'der3', 'der4', 'der5', 'der6', 'der7', 'der8', 'der9', 'der10',
       'der11', 'der12', 'der13', 'der14', 'der15', 'der16', 'der17', 'der18',
       'der19', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8',
       'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'target'],
      dtype='object')

In [0]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    x_resampled, y_resampled, test_size=0.7, random_state=42)
cl.fit(x_train,y_train)

Prediction and Scores

In [0]:
y_pred=cl.predict(x_test)
f=confusion_matrix(y_test,y_pred)
print (average_precision_score(y_test,y_pred))
print (precision_recall_curve(y_test,y_pred))
print (f1_score(y_test,y_pred))

In [0]:
tn, fp, fn,tp=confusion_matrix(y_test,y_pred).ravel()




In [0]:
len(train_data)
tn+tp

In [0]:
precision_recall_curve(y_test,y_pred)


(array([1., 1.]), array([1., 0.]), array([1]))

In [0]:
average_precision_score(y_test,y_pred)


1.0

In [0]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Jun  7 22:58:00 2018

@author: gopiprasanth
"""

"""Impute missing values with k nearest classifier."""

import numpy as np
import pandas as pd
from sklearn import neighbors


class Imputer:
    """Imputer class."""

    def _fit(self, X, column, k=10, is_categorical=False):
        """Fit a knn classifier for missing column.
        - Args:
                X(numpy.ndarray): input data
                column(int): column id to be imputed
                k(int): number of nearest neighbors, default 10
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                clf: trained k nearest neighbour classifier
        """
        clf = None
        if not is_categorical:
            clf = neighbors.KNeighborsRegressor(n_neighbors=k)
        else:
            clf = neighbors.KNeighborsClassifier(n_neighbors=k)
        # use column not null to train the kNN classifier
        missing_idxes = np.where(pd.isnull(X[:, column]))[0]
        if len(missing_idxes) == 0:
            return None
        X_copy = np.delete(X, missing_idxes, 0)
        X_train = np.delete(X_copy, column, 1)
        # if other columns still have missing values fill with mean
        col_mean = None
        if not is_categorical:
            col_mean = np.nanmean(X, 0)
        else:
            col_mean = np.nanmedian(X, 0)
        for col_id in range(0, len(col_mean) - 1):
            col_missing_idxes = np.where(np.isnan(X_train[:, col_id]))[0]
            if len(col_missing_idxes) == 0:
                continue
            else:
                X_train[col_missing_idxes, col_id] = col_mean[col_id]
        y_train = X_copy[:, column]
        # fit classifier
        clf.fit(X_train, y_train)
        return clf

    def _transform(self, X, column, clf, is_categorical):
        """Impute missing values.
        - Args:
                X(numpy.ndarray): input numpy ndarray
                column(int): index of column to be imputed
                clf: pretrained classifier
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                X(pandas.dataframe): imputed dataframe
        """
        missing_idxes = np.where(np.isnan(X[:, column]))[0]
        X_test = X[missing_idxes, :]
        X_test = np.delete(X_test, column, 1)
        # if other columns still have missing values fill with mean
        col_mean = None
        if not is_categorical:
            col_mean = np.nanmean(X, 0)
        else:
            col_mean = np.nanmedian(X, 0)
        # fill missing values in each column with current col_mean
        for col_id in range(0, len(col_mean) - 1):
            col_missing_idxes = np.where(np.isnan(X_test[:, col_id]))[0]
            # if no missing values for current column
            if len(col_missing_idxes) == 0:
                continue
            else:
                X_test[col_missing_idxes, col_id] = col_mean[col_id]
        # predict missing values
        y_test = clf.predict(X_test)
        X[missing_idxes, column] = y_test
        return X

    def knn(self, X, column, k=10, is_categorical=False):
        """Impute missing value with knn.
        - Args:
                X(pandas.dataframe): dataframe
                column(str): column name to be imputed
                k(int): number of nearest neighbors, default 10
                is_categorical(boolean): is continuous or categorical feature
        - Returns:
                X_imputed(pandas.dataframe): imputed pandas dataframe
        """
        X, column = self._check_X_y(X, column)
        clf = self._fit(X, column, k, is_categorical)
        if clf is None:
            return X
        else:
            X_imputed = self._transform(X, column, clf, is_categorical)
            return X_imputed

    def _check_X_y(self, X, column):
        """Check input, if pandas.dataframe, transform to numpy array.
        - Args:
                X(ndarray/pandas.dataframe): input instances
                column(str/int): column index or column name
        - Returns:
                X(ndarray): input instances
        """
        column_idx = None
        if isinstance(X, pd.core.frame.DataFrame):
            if isinstance(column, str):
                # get index of current column
                column_idx = X.columns.get_loc(column)
            else:
                column_idx = column
            X = X.as_matrix()
        else:
            column_idx = column
        return X, column_idx
    



#random missing dataframe
        
df = pd.DataFrame({'A':[1,2,3,4],'B':[4,5,6,7],'C':[7,8,9,10]})
                   

print (df)


mask = np.random.choice([True, False], size=df.shape)
print (mask)


mask[mask.all(1),-1] = 0
print (mask)

print (df.mask(mask))
impute=Imputer()
impute.knn(df.mask(mask),1,k=2)
impute.knn(df.mask(mask),1,k=2)
